In [41]:
import pandas as pd 
import json

df = pd.read_csv('mturk_results/Batch_5320889_batch_results_first10_pilot.csv')
df.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.q_strings',
       'Answer.taskAnswers', 'Approve', 'Reject'],
      dtype='object')

In [40]:
submission_df = df[['WorkerId','WorkTimeInSeconds','Answer.taskAnswers']]
submission_df

,WorkerId,WorkTimeInSeconds,Answer.taskAnswers
0,A1WTHBAWHRVU4P,559,"[{""104_B_ours_instag_q1"":{""instag"":false,""ours..."
1,A1G1O82VQI14Y1,189,"[{""074_A_ga_ours_q1"":{""ga"":false,""ours"":true},..."
2,A2DXOYMV4TJWLF,377,"[{""074_B_ours_instag_q1"":{""instag"":false,""ours..."
3,A1IRR3HIAYF363,452,"[{""218_B_ours_instag_q1"":{""instag"":false,""ours..."
4,AXPTF739HYEZX,517,"[{""074_B_talkg_ours_q1"":{""ours"":true,""talkg"":f..."
5,A2T8AISA63GR31,553,"[{""074_A_ours_ga_q1"":{""ga"":false,""ours"":true},..."
6,A25GP17YDFVXPC,198,"[{""074_B_ours_talkg_q1"":{""ours"":true,""talkg"":f..."
7,AIOJAGGDX2E2Z,488,"[{""074_A_ga_ours_q1"":{""ga"":false,""ours"":true},..."
8,A1S1HDM5HBGT1B,405,"[{""104_A_instag_ours_q1"":{""instag"":true,""ours""..."
9,A2NC3DH5SD5OPJ,554,"[{""218_B_ours_ga_q1"":{""ga"":false,""ours"":true},..."


In [49]:
# answer stats per worker
# recall: each worker answers 9 x 3 = 27 questions, full score is 27
for i in range(len(submission_df)):
    print(f"workerid: {submission_df.at[i, 'WorkerId']}, worktime: {submission_df.at[i, 'WorkTimeInSeconds']}")
    answer_json = json.loads(submission_df.at[i, 'Answer.taskAnswers'])
    methods = ['ours', 'ga', 'talkg', 'instag']
    score_dict = {method: 0 for method in methods}
    for question, answer in answer_json[0].items():
        # print(f"question: {question}, answer: {answer}") # see individual answers
        for method, score in answer.items():
            if method in methods:
                score_dict[method] += int(score)
    print(f"score_dict: {score_dict}")




workerid: A1WTHBAWHRVU4P, worktime: 559
score_dict: {'ours': 24, 'ga': 0, 'talkg': 3, 'instag': 0}
workerid: A1G1O82VQI14Y1, worktime: 189
score_dict: {'ours': 8, 'ga': 7, 'talkg': 7, 'instag': 5}
workerid: A2DXOYMV4TJWLF, worktime: 377
score_dict: {'ours': 10, 'ga': 4, 'talkg': 7, 'instag': 6}
workerid: A1IRR3HIAYF363, worktime: 452
score_dict: {'ours': 25, 'ga': 1, 'talkg': 1, 'instag': 0}
workerid: AXPTF739HYEZX, worktime: 517
score_dict: {'ours': 21, 'ga': 4, 'talkg': 2, 'instag': 0}
workerid: A2T8AISA63GR31, worktime: 553
score_dict: {'ours': 24, 'ga': 0, 'talkg': 3, 'instag': 0}
workerid: A25GP17YDFVXPC, worktime: 198
score_dict: {'ours': 18, 'ga': 3, 'talkg': 4, 'instag': 2}
workerid: AIOJAGGDX2E2Z, worktime: 488
score_dict: {'ours': 20, 'ga': 0, 'talkg': 4, 'instag': 3}
workerid: A1S1HDM5HBGT1B, worktime: 405
score_dict: {'ours': 15, 'ga': 3, 'talkg': 3, 'instag': 6}
workerid: A2NC3DH5SD5OPJ, worktime: 554
score_dict: {'ours': 17, 'ga': 3, 'talkg': 3, 'instag': 4}
workerid: A1P

In [57]:
# method score distribution by subj+A/B

# create df (workerid, worktime, subj, speech, methodA, methodB, methodA_wins, methodB_wins, ours_wins, ga_wins, talkg_wins, instag_wins)
df_list = []
for i in range(len(submission_df)):
    workerId = submission_df.at[i, 'WorkerId']
    worktime = submission_df.at[i, 'WorkTimeInSeconds']

    answer_json = json.loads(submission_df.at[i, 'Answer.taskAnswers'])
    methods = ['ours', 'ga', 'talkg', 'instag']
    # score_dict = {method: 0 for method in methods}
    for question, answer in answer_json[0].items():
        subj = question.split('_')[0]
        speech = question.split('_')[1]
        methodA = question.split('_')[2]
        methodB = question.split('_')[3]
        question_id = question.split('_')[4]

        methodA_wins = answer[methodA]
        methodB_wins = answer[methodB]

        # using wins, get 0,1,0,0 for ours, ga, talkg, instag
        ours_wins = 0
        ga_wins = 0
        talkg_wins = 0
        instag_wins = 0
        if methodA == 'ours':
            ours_wins = int(methodA_wins)
        elif methodA == 'ga':
            ga_wins = int(methodA_wins)
        elif methodA == 'talkg':
            talkg_wins = int(methodA_wins)
        elif methodA == 'instag':
            instag_wins = int(methodA_wins)
        
        if methodB == 'ours':
            ours_wins = int(methodB_wins)
        elif methodB == 'ga':
            ga_wins = int(methodB_wins)
        elif methodB == 'talkg':
            talkg_wins = int(methodB_wins)
        elif methodB == 'instag':
            instag_wins = int(methodB_wins)

        # create tuple
        df_list.append((workerId, worktime, subj, speech, question_id, methodA, methodB, methodA_wins, methodB_wins, ours_wins, ga_wins, talkg_wins, instag_wins))

# create df
df = pd.DataFrame(df_list, columns=['WorkerId', 'WorkTimeInSeconds', 'subj', 'speech', 'q_id', 'methodA', 'methodB', 'methodA_wins', 'methodB_wins', 'ours_wins', 'ga_wins', 'talkg_wins', 'instag_wins'])
df

,WorkerId,WorkTimeInSeconds,subj,speech,q_id,methodA,methodB,methodA_wins,methodB_wins,ours_wins,ga_wins,talkg_wins,instag_wins
0,A1WTHBAWHRVU4P,559,104,B,q1,ours,instag,True,False,1,0,0,0
1,A1WTHBAWHRVU4P,559,104,B,q2,ours,instag,True,False,1,0,0,0
2,A1WTHBAWHRVU4P,559,104,B,q3,ours,instag,True,False,1,0,0,0
3,A1WTHBAWHRVU4P,559,218,A,q1,ours,ga,True,False,1,0,0,0
4,A1WTHBAWHRVU4P,559,218,A,q2,ours,ga,True,False,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,A2500R0F1TR1P0,548,306,A,q2,ga,ours,False,True,1,0,0,0
320,A2500R0F1TR1P0,548,306,A,q3,ga,ours,True,False,0,1,0,0
321,A2500R0F1TR1P0,548,460,A,q1,ga,ours,False,True,1,0,0,0
322,A2500R0F1TR1P0,548,460,A,q2,ga,ours,True,False,0,1,0,0


In [63]:
df_binary = df.drop(columns=['methodA_wins', 'methodB_wins', 'methodA', 'methodB'])
df_binary

,WorkerId,WorkTimeInSeconds,subj,speech,q_id,ours_wins,ga_wins,talkg_wins,instag_wins
0,A1WTHBAWHRVU4P,559,104,B,q1,1,0,0,0
1,A1WTHBAWHRVU4P,559,104,B,q2,1,0,0,0
2,A1WTHBAWHRVU4P,559,104,B,q3,1,0,0,0
3,A1WTHBAWHRVU4P,559,218,A,q1,1,0,0,0
4,A1WTHBAWHRVU4P,559,218,A,q2,1,0,0,0
...,...,...,...,...,...,...,...,...,...
319,A2500R0F1TR1P0,548,306,A,q2,1,0,0,0
320,A2500R0F1TR1P0,548,306,A,q3,0,1,0,0
321,A2500R0F1TR1P0,548,460,A,q1,1,0,0,0
322,A2500R0F1TR1P0,548,460,A,q2,0,1,0,0


In [67]:
df_binary.groupby(['WorkerId','WorkTimeInSeconds']).sum(['ours_wins', 'ga_wins', 'talkg_wins', 'instag_wins']).reset_index()

,WorkerId,WorkTimeInSeconds,ours_wins,ga_wins,talkg_wins,instag_wins
0,A1G1O82VQI14Y1,189,8,7,7,5
1,A1IRR3HIAYF363,452,25,1,1,0
2,A1PS4684V34OYQ,499,26,1,0,0
3,A1S1HDM5HBGT1B,405,15,3,3,6
4,A1WTHBAWHRVU4P,559,24,0,3,0
5,A2500R0F1TR1P0,548,12,9,6,0
6,A25GP17YDFVXPC,198,18,3,4,2
7,A2DXOYMV4TJWLF,377,10,4,7,6
8,A2NC3DH5SD5OPJ,554,17,3,3,4
9,A2T8AISA63GR31,553,24,0,3,0
